In [ ]:
#запуск на kaggle

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
from torchvision import transforms

import os
import zipfile

In [3]:
if torch.cuda.is_available():
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev) 

In [4]:
base_dir = "../input/cian-datafest-2019"
train_dir = os.path.join(base_dir, "train.zip")
test_dir = os.path.join(base_dir, "test.zip")

with zipfile.ZipFile(train_dir,"r") as z:
    z.extractall()
    
with zipfile.ZipFile(test_dir,"r") as z:
    z.extractall()

In [5]:
torch.manual_seed(10)
transform = transforms.Compose([transforms.Resize((56,56)), transforms.ToTensor()])

train_data = torchvision.datasets.ImageFolder('./train', 
                                              transform=transform)

trainset = torch.utils.data.Subset(train_data, list(range(0, len(train_data), 6)))

train_set, val_set = torch.utils.data.random_split(trainset, [6000, 3221])
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=500, 
                                                shuffle=True, num_workers=2)
val_data_loader = torch.utils.data.DataLoader(val_set, batch_size=500, 
                                              shuffle=True, num_workers=2)

In [6]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=1, padding=1)
        self.conv_layer2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=4, stride=1)
        self.conv_layer3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=2, stride=2,padding=1)
        self.pooling_layer1 = nn.MaxPool2d(kernel_size=2)
        self.pooling_layer2 = nn.AvgPool2d(kernel_size=2)
        
        self.linear_layer1 = nn.Linear(in_features=1568, out_features=100)
        self.linear_layer2 = nn.Linear(in_features=100, out_features=num_classes)
        
        self.relu = nn.ReLU()
    def forward(self, inputs):
        
        output_1 = self.relu(self.conv_layer1(inputs))
        output_2 = self.pooling_layer1(output_1)
        output_3 = self.relu(self.conv_layer2(output_2))
        output_4 = self.pooling_layer2(output_3)
        output_5 = self.relu(self.conv_layer3(output_4))
        
        output_6 = torch.flatten(output_5, 1)
        output_7 = self.relu(self.linear_layer1(output_6))
        output = self.linear_layer2(output_7)
        
        return output

In [7]:
model = CNN(1).to(device)
print(model)

In [8]:
# Ваш код здесь (loss function + optimizer)
#Adam 0.001
#BCEWithLogitsLoss
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
# обучение
torch.manual_seed(10)

epochs = 5

total_step = len(train_data_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
               .format(epoch+1, epochs, i+1, total_step, loss.item()))

In [10]:
# оценка качества
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_data_loader:
        images = images.to(device)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)
        
        outputs = model(images)
        predicted = (torch.sigmoid(outputs.data) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

In [21]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv_layer2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv_layer3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=2,padding=1)
        self.conv_layer4 = nn.Conv2d(in_channels=256, out_channels=4096, kernel_size=7, stride=1)
        self.pooling_layer1 = nn.MaxPool2d(kernel_size=2)
        self.pooling_layer2 = nn.MaxPool2d(kernel_size=2)
        self.pooling_layer3 = nn.MaxPool2d(kernel_size=2)
        
        self.linear_layer1 = nn.Linear(in_features=4096, out_features=1000)
        self.linear_layer2 = nn.Linear(in_features=1000, out_features=num_classes)
        
        self.relu = nn.ReLU()
    def forward(self, inputs):
        
        output_1 = self.relu(self.conv_layer1(inputs))
        output_2 = self.pooling_layer1(output_1)
        output_3 = self.relu(self.conv_layer2(output_2))
        output_4 = self.pooling_layer2(output_3)
        output_5 = self.relu(self.conv_layer3(output_4))
        output_6 = self.pooling_layer3(output_5)
        output_7 = self.relu(self.conv_layer4(output_5))
        
        output_8 = torch.flatten(output_7, 1)
        nn.Dropout(p=0.5)
        output_9 = self.relu(self.linear_layer1(output_8))
        output = self.linear_layer2(output_9)
        
        return output

In [22]:
model = CNN(1).to(device)
print(model)

In [23]:
# Ваш код здесь (loss function + optimizer)
#Adam 0.001
#BCEWithLogitsLoss
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [24]:
# обучение
torch.manual_seed(10)

epochs = 5

total_step = len(train_data_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
               .format(epoch+1, epochs, i+1, total_step, loss.item()))

In [25]:
# оценка качества
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_data_loader:
        images = images.to(device)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)
        
        outputs = model(images)
        predicted = (torch.sigmoid(outputs.data) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

In [29]:
from torchvision.models import vgg16

In [30]:
vgg16=vgg16(pretrained=True)